# SegmentStyleNotebook

This is a text cell. Start editing!

In [1]:

import java.awt.Color._
import java.awt.image.BufferedImage
import java.awt.{Color, Desktop}
import java.io.File
import java.net.{URI, URL}
import java.util
import java.util.UUID
import java.util.concurrent.Executors
import java.util.concurrent.atomic.AtomicReference
import java.util.stream.DoubleStream
import java.util.zip.ZipFile

import com.fasterxml.jackson.annotation.JsonIgnore
import com.simiacryptus.mindseye.art.models.VGG16
import com.simiacryptus.mindseye.art.ops.{ContentMatcher, GramMatrixEnhancer, MomentMatcher}
import com.simiacryptus.mindseye.art.photo.affinity.RasterAffinity.{adjust, degree}
import com.simiacryptus.mindseye.art.photo.affinity.RelativeAffinity
import com.simiacryptus.mindseye.art.photo.cuda.SmoothSolver_Cuda
import com.simiacryptus.mindseye.art.photo.topology.SearchRadiusTopology
import com.simiacryptus.mindseye.art.photo.{FastPhotoStyleTransfer, SmoothSolver}
import com.simiacryptus.mindseye.art.util._
import com.simiacryptus.mindseye.eval.Trainable
import com.simiacryptus.mindseye.lang.cudnn.CudaSettings
import com.simiacryptus.mindseye.lang.{Coordinate, Tensor}
import com.simiacryptus.mindseye.opt.Step
import com.simiacryptus.mindseye.util.ImageUtil
import com.simiacryptus.notebook.{NotebookOutput, NullNotebookOutput}
import com.simiacryptus.sparkbook.util.Java8Util._
import com.simiacryptus.util.{FastRandom, Util}
import javax.imageio.ImageIO

import scala.collection.mutable.ArrayBuffer
import scala.concurrent.duration.Duration
import scala.concurrent.{Await, ExecutionContext, Future}


@JsonIgnore lazy val fastPhotoStyleTransfer = FastPhotoStyleTransfer.fromZip(new ZipFile(Util.cacheFile(new URI(
  "https://simiacryptus.s3-us-west-2.amazonaws.com/photo_wct.zip"))))

def wct(content: Tensor, style: Tensor, mask: Tensor) = {
  val wctRestyled = fastPhotoStyleTransfer.photoWCT(style, content, DoubleStream.of(mask.getData: _*).average().getAsDouble, 1.0)
  maskedDelta(mask, content, wctRestyled)
}

def maskedDelta(mask: Tensor, base: Tensor, changed: Tensor) = {
  changed.mapCoordsAndFree((c: Coordinate) => {
    val bg = mask.get(c)
    if (bg == 1) changed.get(c)
    else base.get(c)
  })
}

def solver: SmoothSolver = new SmoothSolver_Cuda()

def smoother(content: Tensor) = {
  val topology = new SearchRadiusTopology(content).setSelfRef(true)
  //.setVerbose(true)
  var affinity = new RelativeAffinity(content, topology).setContrast(10).setGraphPower1(2).setMixing(0.2)
    .wrap((graphEdges: util.List[Array[Int]], innerResult: util.List[Array[Double]]) => adjust(graphEdges, innerResult, degree(innerResult), 0.5))
  solver.solve(topology, affinity, 1e-4)
}

def loadImage(file: String, width: Int): BufferedImage = {
  resize(loadTensor(file), width).toImage
}

def resize(tensor: Tensor, width: Int) = {
  Tensor.fromRGB(ImageUtil.resize(tensor.toImage, width, true))
}

def resize(foreground: Tensor, dims: Array[Int]) = {
  Tensor.fromRGB(ImageUtil.resize(foreground.toImage, dims(0), dims(1)))
}

def loadTensor(file: String) = {
  try {
    val read = ImageIO.read(new URL(file.toString))
    if (null == read) throw new IllegalArgumentException("Error reading " + file)
    Tensor.fromRGB(read)
  } catch {
    case e: Throwable =>
      throw new RuntimeException("Error reading " + file, e)
  }
}

def dist(color: Color, x: Seq[Double]) = {
  List(
    color.getRed - x(2).doubleValue(),
    color.getGreen - x(1).doubleValue(),
    color.getBlue - x(0).doubleValue()
  ).map(x => x * x).sum
}

def loadMasks(contentImage: BufferedImage, maskUrl: String, colors: Color*) = {
  val maskTensor = resize(loadTensor(maskUrl), contentImage.getWidth)
  val tensor = Tensor.fromRGB(contentImage)
  try {
    for (clr <- 0 until colors.size) yield {
      tensor.mapCoords((coordinate: Coordinate) => {
        val Array(x, y, c) = coordinate.getCoords()
        val pixelColor = maskTensor.getPixel(x, y)
        val closestColor = colors.zipWithIndex.sortBy(x => dist(x._1, pixelColor)).head
        if (closestColor._2 == clr) {
          tensor.get(coordinate)
        } else {
          0.0
        }
      })
    }
  } finally {
    maskTensor.freeRef()
    tensor.freeRef()
  }
}


Error: object simiacryptus is not a member of package com (936)Error: object simiacryptus is not a member of package com (1311)Error: object simiacryptus is not a member of package com (853)Error: not found: value FastPhotoStyleTransfer (1630)Error: object simiacryptus is not a member of package com (1363)Error: not found: value Util (1673)Error: object fasterxml is not a member of package com (360)Error: not found: type JsonIgnore (1585)Error: object simiacryptus is not a member of package com (412)Error: object simiacryptus is not a member of package com (463)Error: object simiacryptus is not a member of package com (557)Error: object simiacryptus is not a member of package com (642)Error: object simiacryptus is not a member of package com (712)Error: object simiacryptus is not a member of package com (779)Error: object simiacryptus is not a member of package com (981)Error: object simiacryptus is not a member of package com (1030)Error: object simiacryptus is not a member of package